In [1]:
import warnings
warnings.filterwarnings("ignore") 

In [2]:
import pandas as pd 
df=pd.read_csv('C:\\workplace\\electricity_demand\\dataset\\AEP_hourly.csv')
df.sort_values('Datetime',ascending=True,inplace=True)
df.head()

,Datetime,AEP_MW
2183,2004-10-01 01:00:00,12379.0
2184,2004-10-01 02:00:00,11935.0
2185,2004-10-01 03:00:00,11692.0
2186,2004-10-01 04:00:00,11597.0
2187,2004-10-01 05:00:00,11681.0


In [3]:
train_df=df[df['Datetime']<='2017-08-02 23:00:00']
test_df=df[(df['Datetime']>='2017-08-04 00:00:00')]


In [4]:
train_df['month']=pd.to_datetime(train_df['Datetime']).dt.month 
train_df['weekday']=pd.to_datetime(train_df['Datetime']).dt.day_of_week
train_df['hour']=pd.to_datetime(train_df['Datetime']).dt.hour

In [5]:
model_df=train_df.groupby(['month','weekday','hour']).agg({'AEP_MW':'mean'}).reset_index()
model_df.columns=['month', 'weekday', 'hour', 'forecast']
model_df.head()

,month,weekday,hour,forecast
0,1,0,0,15888.758621
1,1,0,1,15482.465517
2,1,0,2,15306.172414
3,1,0,3,15281.362069
4,1,0,4,15378.120690


In [6]:
test_df['month']=pd.to_datetime(test_df['Datetime']).dt.month 
test_df['weekday']=pd.to_datetime(test_df['Datetime']).dt.day_of_week
test_df['hour']=pd.to_datetime(test_df['Datetime']).dt.hour
test_df.columns=['date_time', 'aep_mw','month', 'weekday', 'hour']

In [7]:
pred_df=test_df.merge(model_df,on=['month','weekday','hour'])
pred_df.sort_values('date_time',inplace=True)
pred_df

,date_time,aep_mw,month,weekday,hour,forecast
0,2017-08-04 00:00:00,15045.0,8,4,0,15932.622642
5,2017-08-04 01:00:00,13940.0,8,4,1,14771.000000
9,2017-08-04 02:00:00,13230.0,8,4,2,13993.905660
13,2017-08-04 03:00:00,12624.0,8,4,3,13497.905660
17,2017-08-04 04:00:00,12410.0,8,4,4,13200.339623
...,...,...,...,...,...,...
705,2018-08-02 20:00:00,17673.0,8,3,20,19199.115385
710,2018-08-02 21:00:00,17303.0,8,3,21,19006.807692
715,2018-08-02 22:00:00,17001.0,8,3,22,18678.961538
720,2018-08-02 23:00:00,15964.0,8,3,23,17357.288462


In [8]:
from sklearn.metrics import mean_absolute_percentage_error 
mean_absolute_percentage_error(pred_df['aep_mw'],pred_df['forecast'])

0.08721591567415211

In [11]:
import plotly.graph_objects as go
figure=go.Figure()
figure.add_trace(go.Scatter(x=pred_df['date_time'],y=pred_df['aep_mw'],name='actual_electricity_load'))
figure.add_trace(go.Scatter(x=pred_df['date_time'],y=pred_df['forecast'],name='mlp_forecast'))